<div class="alert alert-block alert-info">
    <h2 align="center"style="color: black;">Machine Learning-based Algorithmic Trading</h2>
    <h3 align="center"style="color: black;">Execution Notebook</h3>
</div>

# 1. Initialization

In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [ ]:
import sys
from dotenv import load_dotenv
import os
load_dotenv()
sys.path.append("./forex_MLOps")

## 1.1 Import Functions

In [ ]:
import uuid
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message="Trying to unpickle estimator StandardScaler from version")
from configs.history_data_crawlers_config import root_path
from datetime import timedelta
from realtime.realtime_dataset_functions import dataset_gen_realtime_loop
run_id = str(uuid.uuid4())

from realtime.realtime_utils import (
    add_RSI_to_realtime_dataset,
    add_real_time_candles,
    add_candle_fe,
    add_fe_cndl_shift_fe_realtime_run,
    add_fe_win_realtime_run,
    add_fe_time_realtime_run,
    add_fe_market_close_realtime_run,
    get_vantage_time_now,
    sleep_until_next_run,
    predict_realtime,
    is_forex_market_open,
)


## 1.2 Initialize Reporter (Telegram)

Configure your telegram bot variables. You need to set `TELEGRAM_CHAT_ID`, `TELEGRAM_CHAT_ID` and `TELEGRAM_BOT_TOKEN_SIGNAL` in the .env file. See [this tutorial](https://gist.github.com/nafiesl/4ad622f344cd1dc3bb1ecbe468ff9f8a).

In [ ]:
from utils.telegram_functions import log_agent
from datetime import datetime
import pytz


bot_token = os.getenv('TELEGRAM_BOT_TOKEN')
bot_token_signal = os.getenv('TELEGRAM_BOT_TOKEN_SIGNAL')
chat_id = os.getenv('TELEGRAM_CHAT_ID')

now = datetime.now(pytz.timezone('Asia/Tehran'))
date_string = now.strftime("%d/%m/%Y %H:%M:%S")
bot = log_agent(bot_token=bot_token,chat_id=chat_id, PRINT=True, TELEGRAM=True)
bot_signal = log_agent(bot_token=bot_token_signal,chat_id=chat_id, PRINT=True, TELEGRAM=True)
bot.send_message(text="🚩"*15)
bot.send_message(text="👇"*15)
bot.send_message(text=f"start date_time ={date_string} - tehran time")  
bot_signal.send_message(text=f"start date_time ={date_string} - tehran time")  

## 1.3 Load Model & Set Strategy

In [ ]:
import xgboost as xgb
import pandas as pd
import pickle
import joblib
from configs.history_data_crawlers_config import root_path

models_list = [
    {
        "model_name": "XGB_USDJPY_1",
        "model_path": f"{root_path}/data/models/XGB_long_USDJPY_M400_TP30_SL7_prof1675.01_max_dd-9.76_median_sig269.00_date2024-10-09_14_40.pkl",
        "target_details":{
            "trade_mode": "long",
            "target_symbol": "USDJPY",
            "look_ahead": 400,
            "take_profit": 30,
            "stop_loss": 7,
        },

        "strategy_details":{
            "trade_mode": "long",
            "target_symbol": "USDJPY",
            "look_ahead": 300,
            "take_profit": 30,
            "stop_loss": 20,
            "volume": 1,
        },

    },

    
]

print(f"No. of models to load: {len(models_list)}")


for model_info in models_list:
    print(f"---> model_id_name: {model_info['model_name']}")
    with open(model_info["model_path"], 'rb') as file:
         model_info["model_object"] = pickle.load(file)


bot.send_message("--> models loaded.")

## 1.4 Set Feature Creation Config

### General Config

In [ ]:
# from configs.feature_configs_general import generate_general_config
# feature_config = generate_general_config()

### Model Specific Config

In [ ]:
df_imp = model_info["model_object"].feature_importance
model_features = set(df_imp.feature_name)
from utils.feature_config_extractor.extract_config_from_features import extract_config_from_selected_feature
feature_config = extract_config_from_selected_feature(model_features)
del feature_config['NOT_SYMBOL']

In [ ]:
feature_config

## 1.5 Initialize Metatrader

In [ ]:
from data_crawlers.metatrader_func import (
  initialize_login_metatrader
)

from trade_execution.metatrader5_trade_execution_functions import (
  get_account_info,
  initialize_symbols,
  place_order_instant,
  get_open_orders,
  get_open_positions,
  close_positions_all_of_them,
  check_active_positions_for_time_force_close,
  cal_candle_time_now,
)

all_trades = {}

mt5 = initialize_login_metatrader()

account_info_dict = get_account_info(mt5,assert_in_demo=True, print_info=True)

assert initialize_symbols(
  mt5,
 ["EURUSD", "USDJPY"]
), "!!! some symbol is not in metatrader for trade"

bot.send_message(text="--> metatrader connected.")

## 1.6 Initialize Dataset

In [ ]:
"""
create dataset for the first time. 

"""

data_size_in_days = 20

fe_functions = [
    add_real_time_candles,
    add_RSI_to_realtime_dataset,
    add_fe_cndl_shift_fe_realtime_run,
    add_candle_fe,
    add_fe_time_realtime_run,
    add_fe_win_realtime_run,
    add_fe_market_close_realtime_run,

]

after_merge_functions = [
  
]

dataset_config = {
    "features": [
        "fe_cndl",
        "fe_RSI",
        "fe_ratio",
        "fe_EMA",
        "fe_SMA",
        "fe_cndl_shift",
        "fe_WIN",
        "fe_ATR",
        "fe_RSTD",
        "fe_time",
        "fe_market_close",
    ]
}

dataset, final_df, crawl_time = dataset_gen_realtime_loop(
    mt5,fe_functions,feature_config, dataset_config, dataset={}, data_size_in_days=data_size_in_days,
)

bot.send_message("--> dataset initialized.")
final_df.index.max()

# 2. Realtime Process

In [ ]:
import pandas as pd
import os

prediction_data_path = f'{root_path}/data/realtime_trade_prediction_data.parquet'  # Replace with your actual file path

try:
  # Attempt to read the DataFrame from the file
  if os.path.exists(prediction_data_path):
    print("--> read realtime_trade_prediction_data:")
    preds_df = pd.read_csv(prediction_data_path)
  else:
    print("--> no realtime_trade_prediction_data file. create an emty one:")
    preds_df = pd.DataFrame()
except Exception as e:
  print(f"An error occurred while reading the file: {e}")
  preds_df = pd.DataFrame()

predictions = {}

In [ ]:
"""
main loop predict

"""
import time as tt
import traceback

run_every = 5 # run every X rounded minute
offset_seconds = 1

bot.send_message(f"--> start the realtime loop, run every {run_every} minuts.")
while True:
    try:
        bot.send_message(f"🏴󠁫󠁩󠁬󠁿 "*5)
        sleep_until_next_run(run_every = run_every , offset_seconds = offset_seconds,reporter=bot)
        assert is_forex_market_open(),"!!! market is close but we are runnig."
        bot.send_message(f"============")

        #?? get data
        dataset, final_df, crawl_time = dataset_gen_realtime_loop(
        mt5,fe_functions,feature_config, dataset_config, dataset=dataset
        )
        bot.send_message(f"--> data updated.")

        #?? predict
        predictions, preds_df = predict_realtime(models_list,predictions,preds_df,crawl_time,final_df,reporter=bot)

        bot.send_message(f"--> model predicted.")
        #?? trade
        candle_now = cal_candle_time_now()
        sinals_df = preds_df.loc[(preds_df["_time"] == candle_now ) & (preds_df["model_prediction"] == 1)].drop_duplicates("model_id_name",keep="last")
        bot.send_message(f"--> shape sinals_df: {sinals_df.shape[0]}.")
        if sinals_df.shape[0]>0:
            bot.send_message(f"--> predict_time: {sinals_df.iloc[0]['predict_time']}.")
            bot_signal.send_message(f"--> predict_time: {sinals_df.iloc[0]['predict_time']}.")
        for _,row in sinals_df.iterrows():
            trade_side = row["strategy_trade_mode"]
            symbol = row["strategy_target_symbol"].upper()
            lot = row["strategy_volume"]
            deviation_points = 20
            tp_points = row["strategy_take_profit"] * 10
            sl_points = row["strategy_stop_loss"] * 10
            look_ahead = row["strategy_look_ahead"] ##?? in minutes
            last_candle_close_price = dataset["st_one"][symbol].loc[dataset["st_one"][symbol]["_time"]==cal_candle_time_now()]["Close"].values[0]

            order_info = place_order_instant(
            mt5 = mt5,
            trade_side = trade_side,
            symbol = symbol,
            lot = float(lot),
            deviation_points = deviation_points,
            tp_points = tp_points,
            sl_points = sl_points,
            last_candle_close_price=last_candle_close_price,
            base_price_mode="candle_close"
            )

            now = get_vantage_time_now()
            order_info["open_position_time"] = now
            order_info["force_close_position_time"] = get_vantage_time_now() + timedelta(minutes=look_ahead) - timedelta(seconds=get_vantage_time_now().second + 1)

            all_trades[int(order_info["result"].order)] = (order_info)
            all_trades[int(order_info["result"].order)]["position_status_active"] = True
            bot.send_message(f"--> position opened. 💲💲💲💲💲")
            bot_signal.send_message(f"--> position opened. 💲💲💲💲💲")

        #?? check for timeing
        all_trades = check_active_positions_for_time_force_close(mt5,all_trades)
        bot.send_message(f"--> func check_active_positions_for_time_force_close done.")
        bot.send_message(f"--> loop end.")
        preds_df.to_parquet(f"{root_path}/data/preds_df.parquet")

    except Exception as e:
        bot.send_message(f"!!! ERROR.")
        bot.send_message(f"{e}.")
        traceback.print_exc()
        tt.sleep(60)
        

# 3. Order Management Functions

In [ ]:

# all_trades = check_active_positions_for_time_force_close(mt5,all_trades)
# # closed_position_info = close_positions_by_id(mt5,order_info["result"].order)
# open_orders = get_open_orders(mt5)
# open_positions = get_open_positions(mt5)
# all_trades,open_orders,open_positions

In [ ]:
"""
close all of the open positions - instant - at market price

"""
# closed_positions = close_positions_all_of_them(mt5)

# closed_positions